<a href="https://colab.research.google.com/github/trefftzc/cis677/blob/main/Intro_to_mpi4py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




# An introduction to mpi4py

Based on the tutorial available at

https://mpi4py.readthedocs.io/en/stable/tutorial.html

mpi4py is not part of the standard set of libraries available in COLAB.
It needs to be installed using pip



In [1]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.2/466.2 kB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.1-cp310-cp310-linux_x86_64.whl size=4266349 sha256=5c534726587399650d2b1060cad0804e44f0c609cc0f4b325c5b9c7ebeb1762f
  Stored in directory: /root/.cache/pip/wheels/3c/ca/13/13218a83854023ccec184e3af482f0f038b434aa32c19afee8
Successfully built mpi4py


Let's start with a small program that illustrates very basic functions in MPI:
- Get_rank
- Get_size

One needs to start with the appropriate import statement

In [2]:
%%writefile example1.py
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
print("Hello world from process ", rank," out of ",size)

Writing example1.py


In COLAB, se need some special additional statements to be able to execute MPI programs.

In [3]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 2 --oversubscribe python example1.py

Hello world from process  0  out of  2
Hello world from process  1  out of  2


We can state that we want more processes to be executed.

In [4]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 4 --oversubscribe python example1.py

Hello world from process  2  out of  4
Hello world from process  3  out of  4
Hello world from process  0  out of  4
Hello world from process  1  out of  4


Sending and receiving messages.

In this example, we will assume that there are two processes, with ids 0 and 1.
Node 0 will send a message to node 1 and then node 1 will answer back.


In [6]:
%%writefile example_send_receive.py
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# First node 0 sends a message to node 1. Node 1 prints the message
if rank == 0:
    msg = "Hello node 1. I am node 0"
    comm.send(msg, dest=1)
elif rank == 1:
    msg = comm.recv(source=0)
    print("I am node 1. This is the msg I received: ",msg)

# Now node 1 sends a message to node 0. Node 0 prints the message
if rank == 1:
    msg = "Hello node 0. I am node 1"
    comm.send(msg, dest=0)
elif rank == 0:
    msg = comm.recv(source=1)
    print("I am node 0. This is the msg I received: ",msg)

Writing example_send_receive.py


In [7]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 2 --oversubscribe python example_send_receive.py

I am node 1. This is the msg I received:  Hello node 1. I am node 0
I am node 0. This is the msg I received:  Hello node 0. I am node 1


The communications on mpi4py are built on top of pickle, a serialization library that allows the sending and receiving of objects. Let's send
integer values instead of strings.

In [8]:
%%writefile example_send_receive_integers.py
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# First node 0 sends the integer 17 to node 1. Node 1 prints the value
if rank == 0:
    msg = 17
    comm.send(msg, dest=1)
elif rank == 1:
    msg = comm.recv(source=0)
    print("I am node 1. This is the msg I received: ",msg)

# Now node 1 sends the integer 34 to node 0. Node 0 prints the value
if rank == 1:
    msg = 34
    comm.send(msg, dest=0)
elif rank == 0:
    msg = comm.recv(source=1)
    print("I am node 0. This is the msg I received: ",msg)

Writing example_send_receive_integers.py


In [10]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 2 --oversubscribe python example_send_receive_integers.py

I am node 1. This is the msg I received:  17
I am node 0. This is the msg I received:  34


Because we are using pickle, we can send objects from one process to the next.

In [14]:
%%writefile example_send_receive_dictionary.py
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# First node 0 sends a dictionary to node 1. Node 1 uses the dictionary
if rank == 0:
    msg = {'a': 65,
           'b': 66,
           'c':67}
    comm.send(msg, dest=1)
elif rank == 1:
    msg = comm.recv(source=0)
    print("I am node 1. I am using the dictionary I received from node 0.",msg['a'])

Overwriting example_send_receive_dictionary.py


In [15]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 2 --oversubscribe python example_send_receive_dictionary.py

I am node 1. I am using the dictionary I received from node 0. 65


We can send numpy arrays:

In [16]:
%%writefile example_sending_numpy_arrays.py
from mpi4py import MPI
import numpy

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# passing MPI datatypes explicitly
if rank == 0:
    data = numpy.arange(1000, dtype='i')
    comm.Send([data, MPI.INT], dest=1, tag=77)
elif rank == 1:
    data = numpy.empty(1000, dtype='i')
    comm.Recv([data, MPI.INT], source=0, tag=77)

# automatic MPI datatype discovery
if rank == 0:
    data = numpy.arange(100, dtype=numpy.float64)
    comm.Send(data, dest=1, tag=13)
elif rank == 1:
    data = numpy.empty(100, dtype=numpy.float64)
    comm.Recv(data, source=0, tag=13)

Writing example_sending_numpy_arrays.py


In [17]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 2 --oversubscribe python example_sending_numpy_arrays.py

Collective Communications are supported in mpi4py.
Let's start with a broadcasting operation.

In [18]:
%%writefile example_broadcasting.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = np.arange(100, dtype='i')
else:
    data = np.empty(100, dtype='i')
comm.Bcast(data, root=0)
for i in range(100):
    assert data[i] == i

Writing example_broadcasting.py


Notice that we have 4 processes in this execution.
If everything goes well, there should be no messages.
The assert statement will fail if the condition is not true. Then a message is produced. The assert statements are being executed on all processes.

In [20]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 4 --oversubscribe python example_broadcasting.py

Now an example of scatter.
In this example, node 0 scatters an array with 16 integers over two recipients.
Each of the recipients will received half of the original array that 0 send out.

In [31]:
%%writefile example_scatter.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

sendbuf = None
if rank == 0:
    sendbuf = np.arange(16, dtype='i')
    print("I am node: ",rank," and this is sendbuf: ",sendbuf)
recvbuf = np.zeros(8, dtype='i')
comm.Scatter(sendbuf, recvbuf, root=0)
print("I am node: ",rank," and this is recvbuf after the scatter operation: ",recvbuf)

Overwriting example_scatter.py


In [32]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 2 --oversubscribe python example_scatter.py

I am node:  0  and this is sendbuf:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
I am node:  0  and this is recvbuf after the scatter operation:  [0 1 2 3 4 5 6 7]
I am node:  1  and this is recvbuf after the scatter operation:  [ 8  9 10 11 12 13 14 15]


And now an example designed to work with 4 participating nodes.
Here every receiving node receives 1/4 of the original array.

In [33]:
%%writefile example_scatter2.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

sendbuf = None
if rank == 0:
    sendbuf = np.arange(16, dtype='i')
    print("I am node: ",rank," and this is sendbuf: ",sendbuf)
recvbuf = np.zeros(4, dtype='i')
comm.Scatter(sendbuf, recvbuf, root=0)
print("I am node: ",rank," and this is recvbuf after the scatter operation: ",recvbuf)

Writing example_scatter2.py


In [35]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 4 --oversubscribe python example_scatter2.py

I am node:  0  and this is sendbuf:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
I am node:  0  and this is recvbuf after the scatter operation:  [0 1 2 3]
I am node:  2  and this is recvbuf after the scatter operation:  [ 8  9 10 11]
I am node:  1  and this is recvbuf after the scatter operation:  [4 5 6 7]
I am node:  3  and this is recvbuf after the scatter operation:  [12 13 14 15]


Now an example of gather. Again we start with 2 nodes.

In [39]:
%%writefile example_gather.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

sendbuf = np.zeros(8, dtype='i') + rank
print("I am node: ",rank," and this is sendbuf: ",sendbuf)
recvbuf = None
if rank == 0:
    recvbuf = np.empty(16, dtype='i')
comm.Gather(sendbuf, recvbuf, root=0)
if rank == 0:
  print("I am node 0 and this is the receive buffer: ",recvbuf)

Overwriting example_gather.py


In [40]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 2 --oversubscribe python example_gather.py

I am node:  0  and this is sendbuf:  [0 0 0 0 0 0 0 0]
I am node:  1  and this is sendbuf:  [1 1 1 1 1 1 1 1]
I am node 0 and this is the receive buffer:  [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]


Next an example that should work with four processes.

In [43]:
%%writefile example_gather2.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

sendbuf = np.zeros(4, dtype='i') + rank
print("I am node: ",rank," and this is sendbuf: ",sendbuf)
recvbuf = None
if rank == 0:
    recvbuf = np.empty(16, dtype='i')
comm.Gather(sendbuf, recvbuf, root=0)
if rank == 0:
  print("I am node 0 and this is the receive buffer: ",recvbuf)

Overwriting example_gather2.py


In [45]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 4 --oversubscribe python example_gather2.py

I am node:  2  and this is sendbuf:  [2 2 2 2]
I am node:  0  and this is sendbuf:  [0 0 0 0]
I am node:  3  and this is sendbuf:  [3 3 3 3]
I am node:  1  and this is sendbuf:  [1 1 1 1]
I am node 0 and this is the receive buffer:  [0 0 0 0 1 1 1 1 2 2 2 2 3 3 3 3]


Reductions are available as well:

In [46]:
%%writefile example_reduction.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

value_to_reduce = rank

result_of_reduction = comm.reduce(value_to_reduce, op=MPI.SUM, root=0)
if rank == 0:
  print("I am node 0 and this is the result of the reduction: ",result_of_reduction)

Writing example_reduction.py


In [47]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 4 --oversubscribe python example_reduction.py

I am node 0 and this is the result of the reduction:  6
